In [ ]:
%%bash
git commit -a -m "checkpoint"

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#パッケージのimport" data-toc-modified-id="パッケージのimport-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>パッケージのimport</a></div><div class="lev1 toc-item"><a href="#namedtuple" data-toc-modified-id="namedtuple-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>namedtuple</a></div><div class="lev1 toc-item"><a href="#定数の設定" data-toc-modified-id="定数の設定-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>定数の設定</a></div><div class="lev1 toc-item"><a href="#経験を保存するメモリクラスを定義します" data-toc-modified-id="経験を保存するメモリクラスを定義します-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>経験を保存するメモリクラスを定義します</a></div><div class="lev2 toc-item"><a href="#sandbox" data-toc-modified-id="sandbox-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>sandbox</a></div><div class="lev1 toc-item"><a href="#brainstorming" data-toc-modified-id="brainstorming-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>brainstorming</a></div><div class="lev1 toc-item"><a href="#Brain" data-toc-modified-id="Brain-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Brain</a></div><div class="lev1 toc-item"><a href="#Agent" data-toc-modified-id="Agent-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Agent</a></div><div class="lev1 toc-item"><a href="#pricer" data-toc-modified-id="pricer-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>pricer</a></div><div class="lev2 toc-item"><a href="#class-version-zero" data-toc-modified-id="class-version-zero-81"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>class version zero</a></div><div class="lev1 toc-item"><a href="#replacing-step()-and-state-initialization" data-toc-modified-id="replacing-step()-and-state-initialization-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>replacing step() and state initialization</a></div><div class="lev2 toc-item"><a href="#sandbox" data-toc-modified-id="sandbox-91"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>sandbox</a></div><div class="lev1 toc-item"><a href="#Environment改（大幅改修した）" data-toc-modified-id="Environment改（大幅改修した）-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Environment改（大幅改修した）</a></div><div class="lev2 toc-item"><a href="#Version-1" data-toc-modified-id="Version-1-101"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Version 1</a></div><div class="lev3 toc-item"><a href="#To-be-determined" data-toc-modified-id="To-be-determined-1011"><span class="toc-item-num">10.1.1&nbsp;&nbsp;</span>To be determined</a></div><div class="lev3 toc-item"><a href="#implementation" data-toc-modified-id="implementation-1012"><span class="toc-item-num">10.1.2&nbsp;&nbsp;</span>implementation</a></div><div class="lev1 toc-item"><a href="#参考（書籍のoriginal-Environment）" data-toc-modified-id="参考（書籍のoriginal-Environment）-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>参考（書籍のoriginal Environment）</a></div>


** 5.3、5.4  PyTorchでDQN **

# パッケージのimport

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

# namedtuple

本コードでは、namedtupleを使用します。

namedtupleを使うことで、値をフィールド名とペアで格納できます。

すると値に対して、フィールド名でアクセスできて便利です。

https://docs.python.jp/3/library/collections.html#collections.namedtuple

以下は使用例です

In [2]:
from collections import namedtuple

Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('名前Aです', 100)

print(Tr_object)  # 出力：tr(name_a='名前Aです', value_b=100)
print(Tr_object.value_b)  # 出力：100

tr(name_a='名前Aです', value_b=100)
100


In [3]:
Tr_object.name_a

'名前Aです'

In [4]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple(
    'Transition', ('s', 't', 'action', 'next_s' , "next_t", 'reward'))


In [5]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple(
    'Transition', ('state', 'action', 'next_state', 'reward'))


# 定数の設定

In [6]:
spot_price = 127.62
volatility = 0.20 # the historical vols or implied vols
dividend_rate =  0.0163
risk_free_rate = 0.001
maturity = 0.1
dt = 0.1


strike_price = 130


#steps = 200
pricer_steps = 100

#dt = maturity / steps

In [7]:
#ENV = 'CartPole-v0'  # 使用する課題名
GAMMA = np.exp(-dt * risk_free_rate)  # 時間割引率
MAX_STEPS = 200  # 1試行のstep数
NUM_EPISODES = 500  # 最大試行回数

# 経験を保存するメモリクラスを定義します

* 概ね書籍のままの実装でいけるはず（？）
* ただし、おそらくサンプル取得時に完全にランダムにしないで3項ツリー的な３つのnext stateをまとめてmini batfchに含めたほうがいいのではないかという気がする

In [8]:
class ReplayMemory:

    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長さ
        self.memory = []  # 経験を保存する変数
        self.index = 0  # 保存するindexを示す変数

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''

        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す

        # namedtupleのTransitionを使用し、値とフィールド名をペアにして保存します
        self.memory[self.index] = Transition(state, action, state_next, reward)

        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)

## sandbox

In [ ]:
tmp = ReplayMemory(3)

In [ ]:
tmp.push(100,0.1, 0 , 101, 0.15 ,0 )

In [ ]:
tmp.push(100,0.1, 0 , 99, 0.15 ,0 )

In [ ]:
tmp.sample(2)

# brainstorming

* 素直な実装ではnum_statesは２となる。（$S$および$t$）
* brainもたぶん書籍のままの実装でいける？？


# Brain

エージェントが持つ脳となるクラスです、DQNを実行します

methodは

* replay: Experience Replayでネットワークの結合パラメータを学習
* decide_action: アクション決定

Q関数をディープラーニングのネットワークをクラスとして定義

In [9]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

In [10]:
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力
        print(self.model.parameters())

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # -----------------------------------------
        # 1. メモリサイズの確認
        # -----------------------------------------
        # 1.1 メモリサイズがミニバッチより小さい間は何もしない
        #print("memory .size : {}".format(len(self.memory)))
        if len(self.memory) < BATCH_SIZE:
            return

        # -----------------------------------------
        # 2. ミニバッチの作成
        # -----------------------------------------
        # 2.1 メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 2.2 各変数をミニバッチに対応する形に変形
        # transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
        # つまり、(state, action, state_next, reward)×BATCH_SIZE
        # これをミニバッチにしたい。つまり
        # (state×BATCH_SIZE, action×BATCH_SIZE, state_next×BATCH_SIZE, reward×BATCH_SIZE)にする
        batch = Transition(*zip(*transitions))

        # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        # 例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
        # それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
        # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        # catはConcatenates（結合）のことです。
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        try:
            non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])
        except:
            print()

        # -----------------------------------------
        # 3. 教師信号となるQ(s_t, a_t)値を求める
        # -----------------------------------------
        # 3.1 ネットワークを推論モードに切り替える
        self.model.eval()

        # 3.2 ネットワークが出力したQ(s_t, a_t)を求める
        # self.model(state_batch)は、右左の両方のQ値を出力しており
        # [torch.FloatTensor of size BATCH_SIZEx2]になっている。
        # ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め
        # それに対応するQ値をgatherでひっぱり出す。
        state_action_values = self.model(state_batch).gather(1, action_batch)

        # 3.3 max{Q(s_t+1, a)}値を求める。ただし次の状態があるかに注意。

        # cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        # 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
        # そしてそのQ値（index=0）を出力します
        # detachでその値を取り出します
        next_state_values[non_final_mask] = self.model(
            non_final_next_states).max(1)[0].detach()

        # 3.4 教師となるQ(s_t, a_t)値を、Q学習の式から求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # -----------------------------------------
        # 4. 結合パラメータの更新
        # -----------------------------------------
        # 4.1 ネットワークを訓練モードに切り替える
        self.model.train()

        # 4.2 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action

# Agent
CartPoleで動くエージェントクラスです、棒付き台車そのものになります

methodは


* 行動価値Qを更新
*  状態を与えると行動を決定
*  状態、選択するアクション、次の状態、報酬などを記憶

In [11]:
class Agent:
    def __init__(self, num_states, num_actions):
        '''課題の状態と行動の数を設定する'''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成

    def update_q_function(self):
        '''Q関数を更新する'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''行動を決定する'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        self.brain.memory.push(state, action, state_next, reward)

# pricer

In [12]:
import QuantLib as ql 

## class version zero

In [14]:
class cl_am_call:
    def price(self , spot_price, strike_price, maturity):
        
        dummy_strike = strike_price / spot_price
        

        option_type = ql.Option.Call
        payoff = ql.PlainVanillaPayoff(option_type, dummy_strike)
        
        maturity_date = self.calculation_date + int(365.0 * maturity)
        settlement = self.calculation_date
        am_exercise = ql.AmericanExercise(settlement, maturity_date)
        
        american_option = ql.VanillaOption(payoff, am_exercise)
        american_option.setPricingEngine(self.binomial_engine)
        ql.Settings.instance().evaluationDate = self.calculation_date

        return (american_option.NPV() * spot_price)
        

    def __init__(self , volatility , dividend_rate , risk_free_rate  ,steps):
        day_count = ql.Actual365Fixed()
        #calendar = ql.UnitedStates()
        calendar = ql.Japan()
        self.calculation_date = ql.Date(8, 5, 2015)
        dummy_spot = 1
        
        self.spot_handle = ql.QuoteHandle(ql.SimpleQuote(1.0))

        ql.Settings.instance().evaluationDate = self.calculation_date


        self.flat_ts = ql.YieldTermStructureHandle(
            ql.FlatForward(self.calculation_date, risk_free_rate, day_count)
        )

        self.dividend_yield = ql.YieldTermStructureHandle(
            ql.FlatForward(self.calculation_date, dividend_rate, day_count)
        )

        #### volatility

        self.flat_vol_ts = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(self.calculation_date, calendar, volatility, day_count)
        )

        #### BS framework

        self.bsm_process = ql.BlackScholesMertonProcess(self.spot_handle, 
                                                   self.dividend_yield, 
                                                   self.flat_ts, 
                                                   self.flat_vol_ts)


        self.binomial_engine = ql.BinomialVanillaEngine(self.bsm_process, "crr", steps)
        



In [15]:
am_call = cl_am_call(volatility ,  dividend_rate ,  risk_free_rate , pricer_steps)

In [ ]:
am_call.price(spot_price, strike_price, maturity)

In [ ]:
am_call.price(1 , 100, 0.1)

# replacing step() and state initialization

* initializationをどうするかな・・


In [16]:
class  decision2next_state:
    
    def __init__(self ,  S0 , volatility , dividend_rate , risk_free_rate  , dt , Maturity):
        self.S0 = S0
        self.volatility = volatility
        self.dividend_rate = dividend_rate
        self.risk_free_rate = risk_free_rate
        self.dt = dt
        self.Maturity = Maturity
        
        self.barbeta_dt = (risk_free_rate - dividend_rate - volatility**2 * 0.5) * dt
        self.sd = volatility * np.sqrt(dt)
    
    def reset(self):
        T0 = 0    
        if True:
            state0 = np.array([self.S0 , T0])
        else :
            state0 = torch.Tensor(np.array([[self.S0 , T0]]))
        return state0
        
    
    def step(self , state , action):
             #stateはtorch.tensor torch.Size([1, 2])
            if False:
                #next_state = state.copy()
                S = state[0]
                T = state[1]
            else:               
                #next_state = state.clone()
                S = state[0][0].item()
                T = state[0][1].item()    
            T_next = T + self.dt
            S_return = self.barbeta_dt + self.sd * np.random.randn()  
            
            S_next = S * np.exp(S_return)
            if True:
                state_next =np.array([S_next , T_next])
            else:
                state_next = torch.Tensor(np.array([[S_next , T_next]]))
            done = (action == 1) or (T_next > self.Maturity) 
            return state_next , done
    

## sandbox

In [ ]:
my_stepper = decision2next_state(spot_price, volatility , dividend_rate , risk_free_rate , 0.1 , 1)

In [ ]:
tmp_state = torch.Tensor(np.array([[100 , 0]]))

In [ ]:
tmp_state = np.array([100,0])

In [ ]:
type(tmp_state)

In [ ]:
my_stepper.step(tmp_state , 0)

In [ ]:
my_stepper.reset()

# Environment改（大幅改修した）
CartPoleを実行する環境のクラスです

* for loopは二重になっている
* 外側のループはEPISODEに関して
    * Episode = 試行：一回ポールを立てて倒れるか200ステップ経過するまでを１エピソードと数える
    * 内側のループはステップに関して
        * 初期状態のBrainを使って、1ステップ目から左右にコントロールしていくことからスタート
        * 各ステップごとに状態と遷移を記録する。
        * 同様に各ステップごとに行動価値関数をアップデートしていく
    * 10エピソード連続で200ステップまで持ちこたえられたら成功
    

## Version 1
* まずはMCパスを発生させて、episode $\approx$ pathであるような形でやってみよう
* 後々、MCパスはtrinomial tree上のみを推移するようにするかもしれないが、ここでは素直にMCパスを普通に発生させてみよう
    * MCパス発生はquantlibにやらせてもいいが自分で実装してしまってもいいかな
* env.stepを
```
 1 time step推進
```
に置き換える。


### To be determined
* QLはここに取り込む？ $\Rightarrow$ maybe yes
* MCもここで？ $\Rightarrow$ maybe yes
    * gymの場合は時間推進はgymが面倒見てくれていた。そのgymはEnvironmentクラスのメンバーになっている。
* 書籍にあったような20回連続でみたいな終了判定基準はもはや適切ではない。ではどのような終了判定基準が良いか
```
現状では正解がわかっているのでいろいろズルをしよう。例えば、10パス連続で最適行使の判定を正解できたときetc
```
* reward設計
```
行使した場合にはrewardを払って行動価値関数がゼロのnext stateに飛ぶ
```
とする.

In [ ]:
%%bash
git commit -a -m "periodical"


### implementation

In [ ]:
tmp = torch.Tensor(np.array([[1,2]]))

In [ ]:
tmp.item()

In [ ]:
np.random.randn()

import QuantLib as ql 

In [ ]:
class myEnvironment:

    def __init__(self, S0 , vol , q , r , K , T , dt ,pricer_steps):
        #self.env = gym.make(ENV)  # 実行する課題を設定
        self.S0 = S0
        self.vol = vol
        self.q = q
        self.r = r
        self.K = K
        self.T = T
        self.dt = dt
        self.stepper = decision2next_state(S0 , vol , q , r , dt, T)
        
        num_states = 2# S and t
        num_actions = 2 # exercise or hold 
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成
        self.pricer = cl_am_call(vol ,  q ,  r , steps = pricer_steps)

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        #complete episodesを終了条件にするのはもはや適切ではない
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            #observation = self.env.reset()  # 環境の初期化
            observation = self.stepper.reset()

            state = observation  # 観測をそのまま状態sとして使用
            if True:
                #print(type(state))
                state = torch.from_numpy(state).type(torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
                state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                action = self.agent.get_action(state, episode)  # 行動を求める。
                ### * episodeを食わせるのはQ学習の定義を見れば納得できる。ここは書籍のままの
        
                
                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
 
                observation_next , done = self.stepper.step(state , action.item())
                #print("done is {}".format(done))

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納
       

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))
                    if True:
                        exercise_value = state[0][0].item() - self.K
                        reward = torch.FloatTensor([exercise_value])    
                        complete_episodes = complete_episodes + 1  ## * 暫定的な処理
                    
                    else:

                        if step < 195:
                            reward = torch.FloatTensor(
                                [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                            complete_episodes = 0  # 連続成功記録をリセット
                        else:
                            reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                            complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    if True:
                        state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                        state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                try:
                    self.agent.update_q_function()
                except:
                    print("something is wrong")
                    break



                # 終了時の処理
                if done:
                    #print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                    #    episode, step + 1, episode_10_list.mean()))
                    #print("exercise value is {}".format(exercise_value))
                    #print(type(state))

                    error = exercise_value - self.pricer.price(state[0][0].item(), self.K , state[0][1].item())
                    print("S : {} / t : {} / exercise value : {} / error : {}".format(state[0][0].item() , state[0][1].item() , exercise_value , error))
                    break
                    ## ** error 
                                # 観測の更新
                state = state_next
            if episode_final:
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 100:
                #print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする

In [ ]:
# main クラス
cartpole_env = myEnvironment(spot_price , volatility , dividend_rate , risk_free_rate , strike_price , maturity , dt , pricer_steps)

In [ ]:
cartpole_env.run()

# 参考（書籍のoriginal Environment）

In [ ]:
class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                    frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める

                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 動画を保存と描画
                display_frames_as_gif(frames)
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする

In [ ]:
# main クラス
cartpole_env = Environment()
cartpole_env.run()
